# Traversal of graphs

A lot of graph algorithms involve traversing the graph structure. This can be done recursively, but unlike the case with trees, there is not necessarily a natural basis case. Trees have leaves but graphs do not necessarily have that. Graphs can have cyclic structure, so a simple recursion will not work. Instead, we need to keep track of which nodes we have already seen during the traversal, so we only process those we haven't yet processed. A depth first traversal, where we keep track of the nodes we have seen in a list, could look like this:

In [1]:
def depth_first_traversal(graph, v, f, seen = None):
    if seen is None: seen = []
    seen.append(v)
    for w in graph[v]:
        if w not in seen:
            depth_first_traversal(graph, w, f, seen)
    f(v)

Here, the `seen = None` is a way to provide a default value to `seen`. If `seen` is `None`, we create an empty list. Don't try to initialise `seen` with the empty list as a default parameter, `seen = []`. This would make the default value *the same* for all calls where the `seen` parameter is not provided--this would give us a full set when we expected an empty one.

In [2]:
def make_list_graph(n_vertices):
    return [[] for _ in range(n_vertices)]
    
def add_list_edge(graph, source, target):
    if target not in graph[source]:
        graph[source].append(target)

In [3]:
g = make_list_graph(6)

add_list_edge(g, 0, 1)
add_list_edge(g, 0, 5)
add_list_edge(g, 1, 2)
add_list_edge(g, 1, 3)
add_list_edge(g, 2, 4)
add_list_edge(g, 3, 5)
add_list_edge(g, 5, 1)

print(g)

[[1, 5], [2, 3], [4], [5], [], [1]]


In [4]:
depth_first_traversal(g, 0, print)

4
2
5
3
1
0


Testing for membership in a list, however, is a linear time operation, so we want to avoid this. One approach would be to use Python's built-in `set` data structure instead.

In [5]:
def depth_first_traversal_set(graph, v, f, seen = None):
    if seen is None: seen = set()
    seen.add(v)
    for w in graph[v]:
        if w not in seen:
            depth_first_traversal_set(graph, w, f, seen)
    f(v)

We can also be more explicit with the set we represent by using a boolean/bit vector for the `seen` data structure. Then, we can set indices to `True` or `False` depending on whether we have seen them.

In [6]:
def depth_first_traversal_bv(graph, v, f, seen = None):
    if seen is None:
        seen = [False] * len(graph)
        
    seen[v] = True
    for w in graph[v]:
        if not seen[w]:
            depth_first_traversal_bv(graph, w, f, seen)
    f(v)

Except for the different data structure, the flow of the traversal is the same as the previous two implementations.

In [7]:
print("bit vector")
depth_first_traversal_bv(g, 0, print)
print("\nset")
depth_first_traversal_set(g, 0, print)

bit vector
4
2
5
3
1
0

set
4
2
5
3
1
0


For the small graph we have traversed so far, there isn't much difference between the different implementations. It doesn't matter if the search take linear time if we have six nodes at most.

In [8]:
def do_nothing(v):
    pass

%timeit depth_first_traversal(g, 0, do_nothing)
%timeit depth_first_traversal_bv(g, 0, do_nothing)
%timeit depth_first_traversal_set(g, 0, do_nothing)

2.73 µs ± 22.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.28 µs ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.53 µs ± 6.49 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


For a larger graph, however, we see that the linear time membership test makes the list implementation substantially slower than the other two. The bit-vector implementation is somewhat faster than the set implementation.

In [9]:
import numpy as np

def make_random_graph(n, k):
    graph = make_list_graph(n)
    for i in range(n):
        for j in np.random.choice(n, size = k):
            add_list_edge(graph, i, j)
    return graph        

rg = make_random_graph(1000, 50)

%timeit depth_first_traversal(rg, 0, do_nothing)
%timeit depth_first_traversal_set(rg, 0, do_nothing)
%timeit depth_first_traversal_bv(rg, 0, do_nothing)

421 ms ± 634 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
4 ms ± 15.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.65 ms ± 34.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


The `make_random_graph` function here makes a graph with $n$ vertices and sample $k$ random neighbours for each. There might be some overlap with those neighbours, so it doesn't mean that the number of neighbours will be *exactly* $k$, but it is the expected number when $k\ll n$. The graph I simulated has 1000 nodes and each has randomly chosen 50 neighbours, so we would expect most nodes to have 50 neighbours since picking 50 out of a 1000 will not have many duplications.

## Closures

The way I have implemented the traversal functions, we can provide a function that will be called on each node. For just printing the node number, as I have done, that is fine, but for most uses we will want to collect some data from the traversal. There is a quite simple way to do this using nested functions, also known as *closures*. If we define a function like this:

In [12]:
def reachable_nodes(graph, v):
    reachable = set()
    def nested_function_that_does_the_job(w):
        reachable.add(w)
    depth_first_traversal_bv(graph, v, nested_function_that_does_the_job)
    return reachable
reachable_nodes(g, 0)

{0, 1, 2, 3, 4, 5}

the nested function, `nested_function_that_does_the_job` can refer to the `reachable` set we define in the `reachable_nodes` function, and we can modify it when the traversal function calls the function. This is something that is called a *closure*. It is a shortening of *enclosure* and means that the inner function can see the variables defined in the enclosing function. There are other ways to achieve similar effects, but this is how I would do this, and is a common solution. You can simply use the same pattern if you need to carry data structures along through a traversal.

Of course, all nodes in this particular graph are reachable from node zero, so it might be more interesting to see what we get for some of the other nodes:

In [15]:
for v in range(6):
    print(v, ':', reachable_nodes(g, v))

0 : {0, 1, 2, 3, 4, 5}
1 : {1, 2, 3, 4, 5}
2 : {2, 4}
3 : {1, 2, 3, 4, 5}
4 : {4}
5 : {1, 2, 3, 4, 5}


## Exercises

It is time to do some exercises to get familiar with graphs. The first one is very simple if you think carefully about it, and doesn't actually involve any traversal.

### Node degrees in an unoriented graph

Write a function that computes the average degree of the nodes in a graph, that is, add together the number of neighbours each nodes in an undirected graph has and divide by the number of nodes. You only have to implement this for the list representation. The easiest way to do the same computation for the matrix representation is to first translate the matrix representation into the list representation and then do the computation there.

To test this function, you might want to create a modified version of `make_random_graph` that creates an unoriented graph.

### Node degrees in an oriented graph

This exercise is slightly more complicated. In an oriented graph, the number of neighbours a node has is not directly observable from the representation in either representation. You will have to collect both the neighbours you can go to from a node and the neighbours that can reach the node in one step. Still, the exercise is to compute the average node degree.

### Size of the reachable sub-graph

For each node, compute the number of reachable nodes in the graph. Test it on some random graphs.